In [1]:
import psycopg2
conn=psycopg2.connect(database='test', user='ivan', password='charta77')
cur=conn.cursor()

In [2]:
import json
data=json.load(open('../data/ebooks.json'))

In [3]:
conn.rollback()
cur.execute('delete from ebook')

In [4]:
for b in data:
    cur.execute('insert into ebook (doc) values (%s)', (json.dumps(b),))
conn.commit()

In [5]:
%%timeit
cur.execute("""select doc->'title' from ebook where doc @> '{"authors":[{"lastname":"Adams"}]}'::jsonb limit 5""")
cur.fetchall()

The slowest run took 29.50 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 318 µs per loop


In [6]:
%%timeit
cur.execute("""select doc->'title' from ebook where doc->>'title' ilike 'ca%' """)
cur.fetchall()

10 loops, best of 3: 42.2 ms per loop


In [7]:
%%timeit
cur.execute("""select distinct b.id, b.doc->'title', a.value->>'lastname' from ebook b, jsonb_array_elements(b.doc->'authors') a 
where a.value->>'lastname' ilike 'ca%'
""")
list(cur)[:10]

10 loops, best of 3: 102 ms per loop


In [8]:
%%timeit
cur.execute("""select  a.value->>'lastname' as lastname, a.value->>'firstname', count(b.id) as num_books 
from ebook b, jsonb_array_elements(b.doc->'authors') a 
group by a.value order by lastname
""")
list(cur)[:10]

10 loops, best of 3: 132 ms per loop


In [9]:

%%timeit
cur.execute("""select  distinct a.value->>'lastname', a.value->>'firstname' from ebook b, jsonb_array_elements(b.doc->'authors') a 
""")
list(cur)

10 loops, best of 3: 101 ms per loop


In [10]:
def build_search_expr(q):
    def create_and(field):
        return " and ".join(map(lambda v: "unaccent(%s) ilike %s"%(field,v),q))
    q=map(lambda x: x.strip(), q.split())
    q=list(map(lambda x: "unaccent('%%%s%%')"%x,q))
    terms=[create_and("b.doc->>'title'"), create_and("b.doc->>'series'"), 
           create_and("concat(a.value->>'firstname', ' ', a.value->>'lastname')")]
    return ' or '.join(terms)
           
print (build_search_expr('karel capek'))

unaccent(b.doc->>'title') ilike unaccent('%karel%') and unaccent(b.doc->>'title') ilike unaccent('%capek%') or unaccent(b.doc->>'series') ilike unaccent('%karel%') and unaccent(b.doc->>'series') ilike unaccent('%capek%') or unaccent(concat(a.value->>'firstname', ' ', a.value->>'lastname')) ilike unaccent('%karel%') and unaccent(concat(a.value->>'firstname', ' ', a.value->>'lastname')) ilike unaccent('%capek%')


In [11]:
%%timeit
search="karel čapek"
cur.execute("select b.id, b.doc from ebook b, jsonb_array_elements(b.doc->'authors') a where %s limit 5" % \
           build_search_expr(search))
cur.fetchall()

1000 loops, best of 3: 2.13 ms per loop
